# w2v based representation learning
We wish to perform w2v on a randomly generated MDP.

1.  Initialize |S| and |A|.
2.  Generate a Transition Probability Matrix 𝙿. Note: keep a seed fixed to ensure reproducibility.
3. Randomly initialize a reward vector associated with each (s,a) ensuring that r(s,a) ∈ [0,1].
4.  Finally generate the trajectory and save it as a .npy file. Load this file into the dataloader of the w2v_model class
5. Perform w2v
6. Ensure that the initial vector embedding associated with each state as obtained from the weights of the initial layer are saved since we will perform comparison with these initial weights and the ones obtained from w2v. \\
To understand how the layers are trained, check the Q-learning code for any MDP game such as frozen lake etc. This will clarify the Q-learning set-up and how we would go about solving the problem using Q-learning. \\
### Finally, we assume minimality of the MDP i.e. p_{i,j}^t>0 for some t>0 ∀ i,j ∈ [|S|]




### We import all the required packages

In [ ]:
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torch.utils.data import Dataset, DataLoader

In [ ]:
# # connect gdrive here
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


### We compute the MDP parameters and define the list storing the names of each (s,a) pair i.e. each word

In [ ]:
H = 1e6# defining the horizon for the MDP
S = 4 # cardinality of State Space (We will replace this with SxA)
A = 4 # cardinality of the Action Space
SxA = S*A # cardinality of the Product Space of State and Actions

# seeds = np.random.randint(10000, size=10)
seeds = [42]
np.random.seed(seeds[0]) # current seed set to 42

current_state = np.random.randint(S) # initialize the current state randomly
current_action = np.random.randint(A) # initialize the current state randomly

# Now we generate a list of words that we will map onto the S X A space for unique representation of each S X A pair.
# word_space = np.array(['the', 'to', 'of', 'in', 'and', 'he', 'is', 'for', 'on', 'said', 'that', 'has', 'says', 'was', 'have', 'it', 'be', 'are', 'with', 'will', 'at', 'mr', 'from', 'by', 'we', 'been', 'as', 'an', 'his', 'not', 'but', 'they', 'after', 'were', 'had', 'there', 'new', 'this', 'australian', 'australia', 'who', 'people', 'palestinian', 'their', 'two', 'government', 'up', 'us', 'south', 'which', 'year', 'one', 'about', 'out', 'if', 'also', 'more', 'when', 'its', 'into', 'would', 'first', 'against', 'last', 'israeli', 'minister', 'arafat', 'over', 'all', 'afghanistan', 'three', 'united', 'world', 'no', 'or', 'police', 'than', 'before', 'attacks', 'fire', 'day', 'security', 'some', 'states', 'you', 'could', 'them', 'today', 'say', 'now', 'told', 'time', 'any', 'laden', 'very', 'bin', 'just', 'can', 'company', 'what', 'president', 'sydney', 'still', 'man', 'four', 'killed', 'taliban', 'al', 'forces', 'our', 'around', 'days', 'being', 'west', 'old', 'other', 'where', 'so', 'officials', 'test', 'qaeda', 'israel', 'general', 'think', 'next', 'federal', 'per', 'force', 'she', 'cent', 'workers', 'leader', 'yesterday', 'state', 'take', 'under', 'hamas', 'him', 'bank', 'suicide', 'years', 'meeting', 'those', 'back', 'down', 'action', 'made', 'morning', 'commission', 're', 'international', 'pakistan', 'centre', 'city', 'attack', 'group', 'afghan', 'through', 'number', 'well', 'while', 'military', 'members', 'qantas', 'called', 'local', 'five', 'area', 'national', 'week', 'gaza', 'union', 'wales', 'hours', 'including', 'since', 'september', 'another', 'east', 'night', 'north', 'report', 'off', 'should', 'war', 'second', 'these', 'staff', 'between', 'go', 'earlier', 'get', 'six', 'defence', 'islamic', 'further', 'do', 'end', 'months', 'team', 'foreign', 'work', 'areas', 'because', 'going', 'sharon', 'power', 'authority', 'near', 'many', 'died', 'eight', 'only', 'way', 'during', 'india', 'know', 'month', 'northern', 'make', 'good', 'melbourne', 'former', 'air', 'spokesman', 'match', 'claims', 'left', 've', 'metres', 'prime', 'authorities', 'support', 'most', 'peace', 'like', 'osama', 'expected', 'given', 'set', 'saying', 'am', 'ago', 'looking', 'militants', 'come', 'bora', 'tora', 'put', 'place', 'several', 'children', 'fighters', 'found', 'unions', 'christmas', 'injured', 'arrested', 'groups', 'africa', 'troops', 'child', 'river', 'royal', 'meanwhile', 'indian', 'part', 'interim', 'yasser', 'official', 'whether', 'then', 'reports', 'hospital', 'terrorist', 'talks', 'economy', 'senior', 'statement', 'mountains', 'how', 'leaders', 'early', 'third', 'start', 'terrorism', 'don', 'industrial', 'hit', 'public', 'trying', 'family', 'court', 'pay', 'army', 'weather', 'believe', 'million', 'radio', 'both', 'john', 'however', 'adelaide', 'control', 'agreement', 'pressure', 'lead', 'long', 'dr', 'following', 'best', 'chief', 'asked', 'help', 'taken', 'want', 'does', 'better', 'overnight', 'few', 'play', 'high', 'service', 'arrest', 'firefighters', 'australians', 'need', 'services', 'queensland', 'confirmed', 'close', 'labor', 'process', 'house', 'community', 'detainees', 'information', 'came', 'secretary', 'opposition', 'believed', 'williams', 'must', 'won', 'possible', 'brought', 'win', 'nations', 'peter', 'her', 'british', 'did', 'hicks', 'released', 'governor', 'accused', 'shot', 'pentagon', 'held', 'damage', 'much', 'maintenance', 'party', 'took', 'conditions', 'such', 'york', 'without', 'council', 'building', 'violence', 'even', 'eastern', 'return', 'director', 'across', 'asylum', 'hill', 'got', 'cut', 'weekend', 'despite', 'kandahar', 'lot', 'airline', 'armed', 'safety', 'dead', 'change', 'winds', 'economic', 'country', 'home', 'men', 'strip', 'palestinians', 'working', 'trade', 'here', 'waugh', 'fires', 'david', 'lee', 'far', 'december', 'seekers', 'news', 'region', 'cricket', 'anti', 'board', 'monday', 'too', 'captured', 'crew', 'race', 'see', 'strong', 'role', 'call', 'training', 'fighting', 'emergency', 'continuing', 'southern', 'american', 'aircraft', 'offer', 'zinni', 'bush', 'george', 'plans', 'charged', 'industry', 'alliance', 'health', 'used', 'bureau', 'head', 'administration', 'water', 'received', 'rate', 'key', 'act', 'past', 'person', 'line', 'station', 'least', 'strikes', 'final', 'legal', 'known', 'town', 'decision', 'hih', 'large', 'issue', 'hundreds', 'israelis', 'your', 'within', 'use', 'major', 'risk', 'britain', 'leading', 'boat', 'stop', 'captain', 'soldiers', 'downer', 'operations', 'department', 'zealand', 'parliament', 'may', 'airport', 'interest', 'late', 'due', 'latest', 'might', 'pm', 'later', 'series', 'able', 'position', 'half', 'officers', 'kabul', 'homes', 'plane', 'laws', 'behind', 'shane', 'give', 'coast', 'death', 'western', 'taking', 'great', 'remain', 'tomorrow', 'hollingworth', 'right', 'policy', 'network', 'un', 'ahead', 'weapons', 'every', 'my', 'forced', 'event', 'hard', 'along', 'seen', 'deaths', 'concerned', 'same', 'towards', 'territory', 'storm', 'cup', 'victory', 'jobs', 'failed', 'really', 'me', 'bill', 'special', 'campaign', 'washington', 'point', 'already', 'jihad', 'heard', 'side', 'abuse', 'timor', 'flight', 'guilty', 'continue', 'thought', 'life', 'others', 'likely', 'detention', 'woomera', 'envoy', 'bombings', 'details', 'november', 'helicopters', 'sunday', 'weeks', 'situation', 'matter', 'launched', 'warne', 'countries', 'th', 'case', 'according', 'again', 'bichel', 'buildings', 'canyoning', 'capital', 'innings', 'mark', 'middle', 'hour', 'space', 'bus', 'seven', 'important', 'mcgrath', 'bombing', 'member', 'bowler', 'rates', 'cabinet', 'enough', 'move', 'cancer', 'boy', 'run', 'rule', 'warplanes', 'claimed', 'money', 'women', 'justice', 'jail', 'reported', 'african', 'immediately', 'ms', 'forward', 'raids', 'based', 'disease', 'added', 'office', 'guides', 'adventure', 'top', 'asio', 'targets', 'aedt', 'caught', 'blue', 'political', 'commonwealth', 'sure', 'show', 'swiss', 'wants', 'movement', 'evidence', 'mission', 'deal', 'young', 'dispute', 'carried', 'perth', 'own', 'human', 'freeze', 'opened', 'wicket', 'carrying', 'march', 'flying', 'certainly', 'job', 'find', 'border', 'each', 'investigation', 'result', 'break', 'although', 'burning', 'always', 'allegations', 'face', 'prepared', 'ground', 'sector', 'collapse', 'beat', 'using', 'island', 'reached', 'probably', 'proposed', 'full', 'growth', 'crash', 'times', 'planning', 'order', 'financial', 'access', 'become', 'banks', 'wave', 'reserve', 'outside', 'afp', 'bowling', 'senator', 'argentina', 'different', 'management', 'stage', 'needs', 'relations', 'program', 'surrender', 'militia', 'suspected', 'believes', 'caves', 'went', 'rejected', 'ariel', 'declared', 'making', 'alleged', 'travel', 'militant', 'ansett', 'road', 'howard', 'responsibility', 'bid', 'kilometres', 'harrison', 'future', 'energy', 'until', 'thousands', 'soon', 'short', 'organisation', 'sent', 'sex', 'tourists', 'calls', 'clear', 'trees', 'radical', 'killing', 'immigration', 'executive', 'allow', 'serious', 'flights', 'inside', 'fight', 'thursday', 'comes', 'quickly', 'opening', 'himself', 'inquiry', 'drop', 'post', 'getting', 'hewitt', 'lives', 'tried', 'july', 'yacht', 'issues', 'refused', 'destroyed', 'accident', 'anything', 'nine', 'karzai', 'alexander', 'gunmen', 'actually', 'bombers', 'quite', 'sources', 'parties', 'among', 'currently', 'agency', 'ministers', 'meet', 'available', 'warned', 'form', 'ended', 'white', 'rural', 'yet', 'parts', 'response', 'understand', 'open', 'done', 'residents', 'jewish', 'school', 'shortly', 'something', 'rights', 'changes', 'directors', 'terms', 'returned', 'list', 'tanks', 'figures', 'address', 'rise', 'sea', 'attempt', 'television', 'means', 'victoria', 'donald', 'source', 'running', 'try', 'wickets', 'kallis', 'sentence', 'announced', 'wounded', 'decided', 'biggest', 'look', 'offices', 'witnesses', 'happened', 'increase', 'fighter', 'brisbane', 'annual', 'measures', 'circumstances', 'ruddock', 'struck', 'friday', 'virus', 'began', 'rather', 'caused', 'tennis', 'whose', 'strike', 'attorney', 'huge', 'though', 'highway', 'islands', 'suharto', 'commissioner', 'wage', 'negotiations', 'car', 'elected', 'robert', 'runs', 'musharraf', 'period', 'having', 'intelligence', 'crowd', 'sir', 'airlines', 'powell', 'confidence', 'worst', 'business', 'market', 'beginning', 'arrived', 'blake', 'recession', 'big', 'supporters', 'rafter', 'glenn', 'allan', 'field', 'afternoon', 'difficult', 'ensure', 'prevent', 'expect', 'involved', 'gives', 'deputy', 'bomb', 'ses', 'facility', 'appeared', 'followed', 'commanders', 'gave', 'advice', 'step', 'less', 'media', 'total', 'expressed', 'nauru', 'pacific', 'hobart', 'away', 'fact', 'commander', 'tour', 'cost', 'ever', 'll', 'beyond', 'law'])
# We generate state names as s0_0, s0_1, s0_2, s0_3, s1_0 ... with si_j where 1<=i<=|S|-1, 1<=j<=|A|-1
word_space = []
for i in range(S):
    for j in range(A):
        word_space.append('s'+str(i)+'_'+str(j))
    # end for j
# end for i
print("Length of word_space i.e. |S|x|A| = ", len(word_space))
print("List of words: ", word_space)

# Please note that right now the word_space length is only 889, hence if SXA>889, then we can't run the mapping as of now.
# However that's a trivial issue which will be fixed soon (by adding more words from a larger dictionary)

states = np.array(word_space[0:S*A]).reshape(S,A) # this stores the word equivalent of each state (state-action pair).
traj = [states[current_state, current_action]] # stores the states observed in a list that we use to generate the .txt file

P_ini = np.random.randint(10, size=(S,A,S)) # Transition Probability Matrix without normalizing each row
# print(P_ini)

# For a 3-D Probability matrix, we peform the following steps
P = np.empty(P_ini.shape)
for index in range(S):
    Pa = P_ini[index,:,:]
    # print("Initial matrix for transitions:", Pa)
    normalizer = np.sum(Pa, axis=1) # Denominator of each row
    Pa = (Pa.transpose()/normalizer).transpose()
    # print("Normalized Probability matrix for transitions: ", Pa)
    P[index,:,:] = Pa
    # print("\n")

print("Final Probability matrix: ", P)
print("\n")
# P now stores the transition probability matrix for the states given (s,a)

'''
Now to obtain the policy i.e. the probability distribution over |A|
'''
# Now we need to create a probability matrix to choose an action from A given the current state = s.
# P_a = np.random.randint(10, size=(S,A))
P_a = np.random.randint(1, 10, size=(S,A)) # this ensures that each action has a non-zero probability given s
# print("Initial Probability matrix for policy:", P_a)
normalizer = np.sum(P_a, axis=1) # Denominator of each row
P_a = (P_a.transpose()/normalizer).transpose()
print("Normalized Probability matrix for policy: ", P_a)

# We generate random rewards and normalize them to ensure 0<= r(s,a) <=1
rewards = np.random.randint(10, size=(S,A))/10 # this stores the reward corresponding to each state action pair
# print("reward matrix: ", rewards)

# assert False, "Break check!"

# # Now we save the Transition Probabilities for comparison with Linear MDP
# temp_filename_P = "%i_states_%i_actions_transition_probability_matrix" %(S,A)
# temp_filename_a = "%i_states_%i_actions_action_probability_matrix" %(S,A)
# temp_filename_r = "%i_states_%i_actions_rewards_matrix" %(S,A)

# np.save(temp_filename_P, P)
# np.save(temp_filename_a, P_a)
# np.save(temp_filename_r, rewards)

Length of word_space i.e. |S|x|A| =  16
List of words:  ['s0_0', 's0_1', 's0_2', 's0_3', 's1_0', 's1_1', 's1_2', 's1_3', 's2_0', 's2_1', 's2_2', 's2_3', 's3_0', 's3_1', 's3_2', 's3_3']
Final Probability matrix:  [[[0.26923077 0.15384615 0.23076923 0.34615385]
  [0.10526316 0.31578947 0.36842105 0.21052632]
  [0.15789474 0.36842105 0.36842105 0.10526316]
  [0.29411765 0.23529412 0.05882353 0.41176471]]

 [[0.5        0.1        0.4        0.        ]
  [0.40909091 0.22727273 0.36363636 0.        ]
  [0.45       0.1        0.3        0.15      ]
  [0.5        0.125      0.25       0.125     ]]

 [[0.25       0.16666667 0.33333333 0.25      ]
  [0.07692308 0.23076923 0.61538462 0.07692308]
  [0.3        0.26666667 0.3        0.13333333]
  [0.05882353 0.17647059 0.35294118 0.41176471]]

 [[0.33333333 0.         0.5        0.16666667]
  [0.4375     0.1875     0.0625     0.3125    ]
  [0.22727273 0.40909091 0.13636364 0.22727273]
  [0.05       0.45       0.05       0.45      ]]]


Normalized

In [ ]:
# H = 10
for t in tqdm(np.arange(1,H)):
    next_state = np.random.choice(np.arange(S), 1, p = P[current_state,current_action,:]) # to generate index for next state
    current_state = next_state[0]
    next_action = np.random.choice(np.arange(A), 1, p = P_a[current_state,:]) # to generate index for next action
    current_action = next_action[0]
    traj.append(states[current_state, current_action])

# print("Trajectory: ", traj)
# # Now we create the string version of the trajectory to provide as input to the word2vec algo
# trajectory = ' '.join(str(x) for x in traj)

100%|██████████| 999999/999999 [01:17<00:00, 12820.73it/s]


## word2vec code

In [ ]:
'''
Using word2vec as is doesn't work because:
    1. We have a continuous state space
    2. We don't have a way to show same states again. This is a problem in training w2v since we need multiple
    context samples for each word else we can't train the w2v model. Thus, we need a rule to generate several
    positive examples for each state that we see in the trajectory
'''
# # All the imports
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# from collections import Counter
# from torch.utils.data import Dataset, DataLoader

# Define a function to convert trajectories into text

embed_size = S
# Start defining word2vec as provided by ChatGPT-4o
def tokenize_text(text): # This function isn't needed since the trajectory is already in the form of tokens
    return text.lower().split()

def build_vocab(text): # Again we already have a vocabulary hence don't need to use this function directly
    # words = tokenize_text(text)
    # words = text
    # word_counts = Counter(words)
    word_counts = Counter(text)
    vocab = {word: i for i, word in enumerate(word_counts.keys())}
    reverse_vocab = {i: word for word, i in vocab.items()}
    return vocab, reverse_vocab, word_counts

def generate_skipgram_pairs(text, window_size=2): # This function gives the word and context pairs.
    # words = tokenize_text(text)
    words = text
    pairs = []
    for i, target_word in enumerate(words):
        window_start = max(i - window_size, 0)
        window_end = min(i + window_size + 1, len(words))
        for j in range(window_start, window_end):
            if i != j:
                pairs.append((words[i], words[j]))
    return pairs


class Word2VecDataset(Dataset):
    def __init__(self, text, vocab, window_size=2, num_neg_samples=5):
        self.vocab = vocab
        self.data = generate_skipgram_pairs(text, window_size)
        self.num_neg_samples = num_neg_samples
        self.vocab_size = len(vocab)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        target, context = self.data[idx] # How does this idx reflect the actual idx of the word?
        ''' Doesn't the above idx reflect the idx numbered pair instead in pairs instead of the pairs
        corresponding to the word at idx? '''
        target_idx = torch.tensor(self.vocab[target], dtype=torch.long)
        context_idx = torch.tensor(self.vocab[context], dtype=torch.long)

        # Negative Sampling: Random words sampled (excluding actual context word)
        negative_samples = torch.randint(0, self.vocab_size, (self.num_neg_samples,))
        while context_idx in negative_samples:
            negative_samples = torch.randint(0, self.vocab_size, (self.num_neg_samples,))

        return target_idx, context_idx, negative_samples


# Building the skip-gram network
class SkipGramNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embed_size, precomputed_vectors=None):
        super(SkipGramNegativeSampling, self).__init__()

        # Create an embedding layer
        self.embeddings = nn.Embedding(vocab_size, embed_size)

        # Initialize embeddings with precomputed word vectors (if available)
        if precomputed_vectors is not None:
            self.embeddings.weight.data.copy_(torch.tensor(precomputed_vectors, dtype=torch.float32))

        # Output layer (to predict context words)
        self.output_layer = nn.Linear(embed_size, vocab_size, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, target_idx):
        target_vec = self.embeddings(target_idx)  # Get word embeddings
        output = self.output_layer(target_vec)  # Compute logits for context words
        final_output = self.sigmoid(output)  # Apply sigmoid to get probabilities
        return final_output

    def loss_function(self, pos_scores, neg_scores):
        """
        Implements negative sampling loss.
        pos_scores: Scores for true context words
        neg_scores: Scores for sampled negative words
        """
        pos_loss = torch.log(self.sigmoid(pos_scores)).mean()
        neg_loss = torch.log(1 - self.sigmoid(neg_scores)).mean()
        return -(pos_loss + neg_loss)  # Negative log likelihood


# Create the training loop:
def train_model(text, precomputed_vectors, embed_size=embed_size, window_size=2, epochs=10, batch_size=16, lr=0.01, num_neg_samples=5):
    vocab, reverse_vocab, _ = build_vocab(text)
    dataset = Word2VecDataset(text, vocab, window_size, num_neg_samples)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Convert precomputed vectors into a matrix (shape: vocab_size x embed_size)
    word_vector_matrix = np.zeros((len(vocab), embed_size))
    for word, idx in vocab.items():
        # word_vector_matrix[idx] = precomputed_vectors.get(word, np.random.randn(embed_size))  # Use random if not found
        word_vector_matrix[idx] = np.random.randn(embed_size)
        # assert False, "Check above line for the .get() function"

    model = SkipGramNegativeSampling(len(vocab), embed_size, word_vector_matrix)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        total_loss = 0
        for target_idx, context_idx, negative_samples in dataloader:
            optimizer.zero_grad()

            # Forward pass
            predictions = model(target_idx)  # Logits (before sigmoid)
            pos_scores = predictions.gather(1, context_idx.unsqueeze(1)).squeeze(1)  # Get true context scores
            neg_scores = predictions.gather(1, negative_samples).mean(dim=1)  # Get negative samples scores

            # Compute loss
            loss = model.loss_function(pos_scores, neg_scores)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

    return model, vocab # this ensures that we can use the vocab created here

###################################################################################################### Example #######################
# Sample text and precomputed word vectors
# word_vectors = {word: np.random.randn(embed_size) for word in ["the", "quick", "brown", "fox", "jumps", "over", "lazy", "dog"]}  # Dummy vectors
# text = "the quick brown fox jumps over the lazy dog"

# Train model
text = traj
word_vectors = []
model, vocab = train_model(text, word_vectors) # word_vectors corresponds to the precomputed-vectors for each word

# Get the learned embedding for a word
word = "s0_0"
if word in vocab:  # ✅ Ensure word exists in vocab
    word_idx = torch.tensor([vocab[word]], dtype=torch.long)
    updated_embedding = model.embeddings(word_idx).detach().numpy()
    print(f"Updated embedding for '{word}': {updated_embedding}")
else:
    print(f"Word '{word}' not found in vocabulary.")



Epoch 1, Loss: 337783.3059
Epoch 2, Loss: 337732.9812
Epoch 3, Loss: 337743.2871
Epoch 4, Loss: 337740.6112
Epoch 5, Loss: 337762.4215
Epoch 6, Loss: 337800.4208
Epoch 7, Loss: 337736.1856
Epoch 8, Loss: 337770.4150
Epoch 9, Loss: 337772.8981
Epoch 10, Loss: 337753.2215
Updated embedding for 's0_0': [[ 0.04957308 -0.3981189  -0.1694536  -1.9118497 ]]


## Q-learning code starts here

Defining the QLearningAgent

In [1]:
import numpy as np
import random

class QLearningAgent:
    def __init__(self, num_states, num_actions, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.num_states = num_states
        self.num_actions = num_actions
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = np.zeros((num_states, num_actions))  # Initialize Q-table

    def choose_action(self, state):
        """Epsilon-greedy action selection."""
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, self.num_actions - 1)  # Explore
        else:
            return np.argmax(self.q_table[state, :])  # Exploit

    def update_q_value(self, state, action, reward, next_state):
        """Q-learning update rule."""
        best_next_action = np.argmax(self.q_table[next_state, :])  # Greedy action for next state
        td_target = reward + self.gamma * self.q_table[next_state, best_next_action]
        td_error = td_target - self.q_table[state, action]
        self.q_table[state, action] += self.alpha * td_error  # Update Q-table

    # To compute the Optimal Policy of the given MDP
    def get_optimal_policy(self):
        """Extracts the optimal policy after training."""
        optimal_policy = np.argmax(self.q_table, axis=1)
        return optimal_policy

    # Printing the final Q-table obtained
    def print_q_table(self):
        """Prints the learned Q-table for debugging."""
        print("Learned Q-table:")
        print(self.q_table)

    # Printing the optimal policy after completion
    def print_optimal_policy(self):
        """Prints the optimal policy in a readable format."""
        optimal_policy = self.get_optimal_policy()
        print("Optimal Policy (best action for each state):")
        for state in range(self.num_states):
            print(f"State {state}: Take Action {optimal_policy[state]}")


###########################################################
# Defining additional functions useful for policy checking
###########################################################
def check_policy_stability(agent, prev_policy):
    """Returns True if the policy is stable (no change in optimal actions)."""
    current_policy = agent.get_optimal_policy()
    return np.array_equal(prev_policy, current_policy)


Policy stabilization based stopping criteria

In [2]:
prev_policy = agent.get_optimal_policy()
stable_count = 0  # Number of episodes with stable policy
stable_threshold = 10  # Stop if policy remains unchanged for 10 episodes

if check_policy_stability(agent, prev_policy):
    stable_count += 1
else:
    stable_count = 0  # Reset count if policy changes

if stable_count >= stable_threshold:
    print(f"Policy stabilized for {stable_threshold} episodes. Stopping training.")
    break

prev_policy = agent.get_optimal_policy()


NameError: name 'agent' is not defined

Q-value not changing based stopping criteria

In [3]:
def run_q_learning(env, agent, num_episodes=1000, convergence_threshold=1e-4):
    """Train the agent and check for Q-value convergence."""
    prev_q_table = np.copy(agent.q_table)  # Store old Q-table
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, _ = env.step(action)
            agent.update_q_value(state, action, reward, next_state)
            state = next_state
            total_reward += reward

        # Compute max Q-value change
        q_change = np.max(np.abs(agent.q_table - prev_q_table))
        prev_q_table = np.copy(agent.q_table)

        if q_change < convergence_threshold:
            print(f"Converged at Episode {episode+1} with max Q-change: {q_change}")
            break

        if (episode + 1) % 100 == 0:
            print(f"Episode {episode+1}: Max Q-value change = {q_change}")

    print("Training completed!")


Using the above MDP for generating the next state

In [ ]:
seeds = [42]
np.random.seed(seeds[0])


### For Frozen Lake or other gymnasium env

In [ ]:
def run_q_learning(env, agent, num_episodes=1000):
    """Train the agent in the given environment."""
    for episode in range(num_episodes):
        state = env.reset()  # Reset environment at the start of each episode
        done = False
        total_reward = 0

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, _ = env.step(action)  # Interact with environment
            agent.update_q_value(state, action, reward, next_state)
            state = next_state
            total_reward += reward

        if (episode + 1) % 100 == 0:
            print(f"Episode {episode+1}: Total Reward = {total_reward}")

    print("Training completed!")

# Example usage:
# Define an environment following OpenAI Gym API with `reset()` and `step(action)`.
# For example, `env = gym.make("FrozenLake-v1")`
# agent = QLearningAgent(num_states=env.observation_space.n, num_actions=env.action_space.n)
# run_q_learning(env, agent, num_episodes=1000)

# ChatGPT code
Here is an implementation of generating the MDP as an environment with step() as a function.


In [2]:
import numpy as np
import random

# Define the randomly generated MDP parameters
S = 4  # Number of states
A = 4  # Number of actions

# Random transition probabilities (S x A x S)
P = np.random.dirichlet(np.ones(S), size=(S, A))

# Random rewards (S x A)
r = np.random.uniform(-1, 1, (S, A))

class RandomMDP:
    """Simulated environment for a randomly generated MDP."""
    def __init__(self, S, A, P, r):
        self.S = S
        self.A = A
        self.P = P  # Transition probabilities
        self.r = r  # Reward function
        self.state = np.random.randint(0, S)  # Initial state

    def reset(self):
        """Resets the environment and returns the initial state."""
        self.state = np.random.randint(0, S)
        return self.state

    def step(self, action):
        """Performs an action, returning (next_state, reward, done, info)."""
        next_state = np.random.choice(self.S, p=self.P[self.state, action])
        reward = self.r[self.state, action]
        self.state = next_state
        done = False  # Assume episode never ends (for simplicity)
        return next_state, reward, done, {}

class QLearningAgent:
    """Q-learning agent."""
    def __init__(self, num_states, num_actions, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.num_states = num_states
        self.num_actions = num_actions
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = np.zeros((num_states, num_actions))  # Initialize Q-table

    def choose_action(self, state):
        """Epsilon-greedy action selection."""
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, self.num_actions - 1)  # Explore5000
    def update_q_value(self, state, action, reward, next_state):
        """Q-learning update rule."""
        best_next_action = np.argmax(self.q_table[next_state, :])  # Greedy action for next state
        td_target = reward + self.gamma * self.q_table[next_state, best_next_action]
        td_error = td_target - self.q_table[state, action]
        self.q_table[state, action] += self.alpha * td_error  # Update Q-table

    def get_optimal_policy(self):
        """Extracts the optimal policy after training."""
        return np.argmax(self.q_table, axis=1)

def run_q_learning(env, agent, num_episodes=1, convergence_threshold=1e-4, stable_threshold=10):
    """Train the agent and check for Q-value and policy convergence."""
    prev_q_table = np.copy(agent.q_table)  # Store old Q-table
    prev_policy = agent.get_optimal_policy()
    stable_count = 0  # Number of episodes with stable policy
    reward_history = []
    episode_count = 0

    for episode in range(num_episodes):
        state = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, _ = env.step(action)
            agent.update_q_value(state, action, reward, next_state)
            state = next_state
            total_reward += reward

            reward_history.append(total_reward)

            # Compute max Q-value change
            q_change = np.max(np.abs(agent.q_table - prev_q_table))
            prev_q_table = np.copy(agent.q_table)

            # Check Q-value convergence
            if q_change < convergence_threshold:
                print(f"Q-values converged at Episode {episode+1} with max Q-change: {q_change}")
                break

            # Check policy stability
            current_policy = agent.get_optimal_policy()
            if np.array_equal(prev_policy, current_policy):
                stable_count += 1
            else:
                stable_count = 0  # Reset count if policy changes

            prev_policy = current_policy

            if stable_count >= stable_threshold:
                print(f"Policy stabilized for {stable_threshold} episodes. Stopping training at Episode {episode+1}.")
                break
            
            episode_count+=1
            if episode_count>1000:
                break

    print("Training completed!")
    return agent, reward_history

# Create a random MDP environment
env = RandomMDP(S, A, P, r)

# Create and train the Q-learning agent
agent = QLearningAgent(num_states=S, num_actions=A)
trained_agent, reward_history = run_q_learning(env, agent)

# Display learned Q-table and optimal policy
import pandas as pd
# import ace_tools as tools

q_table_df = pd.DataFrame(trained_agent.q_table, columns=[f'Action {a}' for a in range(A)])
q_table_df.index.name = 'State'

policy_df = pd.DataFrame({'Optimal Action': trained_agent.get_optimal_policy()})
policy_df.index.name = 'State'

# Display results
# tools.display_dataframe_to_user(name="Learned Q-Table", dataframe=q_table_df)
# tools.display_dataframe_to_user(name="Optimal Policy", dataframe=policy_df)


ValueError: 'p' must be 1-dimensional

In [ ]:
print("Q - Table: ", trained_agent.q_table)
print("Optimal Policy: ", trained_agent.get_optimal_policy())